In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
from squadro.core.benchmarking import benchmark_agents

results = {}

# 5 games, 3 sec per move
results['human'] = {
    'random': 1 / 1,
    'advancement': 1 / 1,
    'relative_advancement': 1 / 1,
    'ab_relative_advancement': 4 / 5,
    'mcts_rollout': 0 / 5,
    'mcts_advancement': 2 / 5,
    'mcts_deep_q_learning': 1 / 5,
}

results = benchmark_agents([
    'mcts_deep_q_learning',
    'mcts_advancement',
    'mcts_rollout',
    'ab_relative_advancement',
    'ab_relative_advancement',
    'relative_advancement',
    'advancement',
    'random',
],
    # n_games=4,
    # max_time_per_move=.01,
    results=results,
)
results

In [ ]:
# dump_pickle(results, 'results.pkl')
# df = pd.read_csv('results_final.csv', index_col=0)
df = pd.DataFrame(results).T
df.insert(0, column=df.index[0], value=[pd.NA] * len(df))
df = df[df.index]
df.to_csv('results.csv')
df

In [ ]:
df = df.infer_objects(copy=False).fillna(0)
df_T = pd.DataFrame(np.tril(1 - df.T, k=-1), columns=df.columns, index=df.index)
df = df + df_T
for i in range(len(df)):
    df.iloc[i, i] = pd.NA
df

In [ ]:
try:
    print(df.to_markdown())
except Exception as e:
    print(e)

In [ ]:
df.columns = df.columns.str.replace('_', ' ')
df.index = df.index.str.replace('_', ' ')
plt.figure(
    figsize=(8, 8)
)
sns.heatmap(df,
            annot=True,  # Show values in cells
            fmt='.2f',  # Format annotations to 2 decimal places
            cmap=['red', 'green'],
            square=True,  # Make cells square
            cbar_kws={'label': 'Values'},  # Customize colorbar
            linewidths=0.5,  # Add gridlines
            center=0.5,
            annot_kws=dict(weight='bold', size=12),
            )  # Center the colormap
plt.title('Pairwise Algorithm Evaluation', weight='bold', size=18)  # Bold title

plt.xticks(weight='bold', size=12)
plt.yticks(weight='bold', size=12)
# plt.tight_layout()
plt.savefig('benchmark.png',bbox_inches='tight', dpi=300)
plt.show()
beats = df > .5
for i in range(len(beats)):
    beats.iloc[i, i] = True
beats.sum(1)

In [ ]:
best = beats.sum(1).idxmax()
print('Best model:', best)